In [1]:
import syft as sy
import torch
from tools import models, datasets
import numpy as np
import pandas as pd
import opacus
from tools.utils import train, test

In [2]:
train_ds, test_ds, val_ds = datasets.Loader.load_MedNIST(sample_size=0.04, test_size=0.0872, val_size=0.125)
train_data, train_labels = train_ds.as_tensor()
test_data, test_labels = test_ds.as_tensor()
val_data, val_labels = val_ds.as_tensor()

print(len(train_data))
print(len(test_data))
print(len(val_data))

1858
206
294


In [3]:
# Constants for tracking purposes
MODEL = 'Deep2DNet'
DATASET = 'MedNIST'
TRACKING = True # Whether or not this run should be tracked in the results.csv
DP = True # Whether or not Differential Privacy should be applied

BATCH_SIZE = 100
EPOCHS = 30

DELTA = 0.0001 # Set to be less then the inverse of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)

# Parameters for training
length = len(train_data)
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE

In [4]:
def run(learning_rate, noise_multiplier, max_grad_norm):   
    # Getting model
    model = models.Deep2DNet(torch)

    # Setting device to train on
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        device = torch.device('cuda:0')
        model.cuda(device)
    else:
        device = torch.device('cpu')
        model.cpu()

    # Optimizer and Loss Function
    params = model.parameters()
    optim = torch.optim.Adam(params=params, lr=learning_rate)
    loss_function = torch.nn.CrossEntropyLoss()

    # Setting up Differential Privacy Engine
    if DP:
        privacy_engine = opacus.privacy_engine.PrivacyEngine(
            model.real_module, sample_rate=SAMPLE_RATE,
            noise_multiplier=noise_multiplier, max_grad_norm=max_grad_norm
        )
        privacy_engine.attach(optim)
    else:
        privacy_engine = None

    # Training
    losses, test_accs, test_losses, epsilons, alphas, epoch_times = train(BATCH_SIZE, EPOCHS, DELTA,
                                                                          model, torch,
                                                                          optim, loss_function, 
                                                                          train_data, train_labels, 
                                                                          test_data, test_labels, 
                                                                          [1, 64, 64], device, privacy_engine)

    # Validation
    val_acc, val_loss = test(model, loss_function, torch, val_data, val_labels, device)

    print(f'Validation Accuracy: {val_acc} ---- Validation Loss: {val_loss}')

    # Tracking all interesting variables and results in .csv file
    if TRACKING:
        d = {
            'model': MODEL,
            'dataset': DATASET,
            'batch_size': BATCH_SIZE,
            'epochs': EPOCHS,
            'learning_rate': learning_rate,
            'train_sample_size': SAMPLE_SIZE,
            'test_sample_size': len(test_data),
            'val_sample_size': len(val_data),
            'delta': DELTA,
            'noise_multiplier': noise_multiplier,
            'max_grad_norm': max_grad_norm,
            'dp_used': DP,
            'epsilons': epsilons,
            'alphas': alphas,
            'train_losses': losses,
            'test_accs': test_accs,
            'test_losses': test_losses,
            'val_acc': val_acc,
            'val_loss': val_loss,
            'epoch_times': epoch_times
        }      
        df = pd.read_csv('./Results/1DS.csv')
        df = df.append(d, ignore_index=True)
        df.to_csv('./Results/1DS.csv', index=False)

In [5]:
import itertools

def gridsearch(lrs, noises, norms):
    grid = itertools.product(lrs, noises, norms)
    num = 0
    
    if DP:
        for learning_rate, noise_multiplier, max_grad_norm in grid:
            num += 1
            print(f'################################# RUN No. {num} #################################')
            run(learning_rate, noise_multiplier, max_grad_norm)
            
    else:
        for learning_rate in lrs:
            num += 1
            print(f'################################# RUN No. {num} #################################')
            run(learning_rate, 0 , 0)
            
    
lrs = [0.003, 0.001, 0.002, 0.0025, 0.0015]
noises = [0.1, 0.5, 1.0, 1.5, 2.0]
norms = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]

gridsearch(lrs, noises, norms)

################################# RUN No. 1 #################################


A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.


###### Epoch 1 ######


Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(ε = 525.69, δ = 0.0001) for α = 1.1
Test Accuracy: 0.6019417475728155 ---- Test Loss: 1.5805317163467407
Epoch time: 2.3664791584014893 seconds
###### Epoch 2 ######
(ε = 959.28, δ = 0.0001) for α = 1.1
Test Accuracy: 0.5679611650485437 ---- Test Loss: 2.263376235961914
Epoch time: 1.0149250030517578 seconds
###### Epoch 3 ######
(ε = 1392.86, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8495145631067961 ---- Test Loss: 1.7165954113006592
Epoch time: 1.0162010192871094 seconds
###### Epoch 4 ######
(ε = 1826.45, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8349514563106796 ---- Test Loss: 1.4040260314941406
Epoch time: 1.018928050994873 seconds
###### Epoch 5 ######
(ε = 2260.04, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8980582524271845 ---- Test Loss: 1.2465990781784058
Epoch time: 1.0149221420288086 seconds
###### Epoch 6 ######
(ε = 2693.62, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8932038834951457 ---- Test Loss: 1.7029997110366821
Epoch time: 1.0169241428375244 seconds
###### Epoch 7 ###

...

(ε = 0.73, δ = 0.0001) for α = 20.0
Test Accuracy: 0.1796116504854369 ---- Test Loss: 1.7860627174377441
Epoch time: 1.0239310264587402 seconds
###### Epoch 2 ######
(ε = 0.95, δ = 0.0001) for α = 17.0
Test Accuracy: 0.1796116504854369 ---- Test Loss: 1.7847027778625488
Epoch time: 1.0229301452636719 seconds
###### Epoch 3 ######
(ε = 1.12, δ = 0.0001) for α = 15.0
Test Accuracy: 0.1796116504854369 ---- Test Loss: 1.7761048078536987
Epoch time: 1.026179313659668 seconds
###### Epoch 4 ######
(ε = 1.27, δ = 0.0001) for α = 14.0
Test Accuracy: 0.15048543689320387 ---- Test Loss: 1.7518556118011475
Epoch time: 1.0262105464935303 seconds
###### Epoch 5 ######
(ε = 1.40, δ = 0.0001) for α = 13.0
Test Accuracy: 0.22330097087378642 ---- Test Loss: 1.7148466110229492
Epoch time: 1.0279872417449951 seconds
###### Epoch 6 ######
(ε = 1.52, δ = 0.0001) for α = 12.0
Test Accuracy: 0.3300970873786408 ---- Test Loss: 1.6602991819381714
Epoch time: 1.0239388942718506 seconds
###### Epoch 7 ######
(ε 

A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(ε = 0.73, δ = 0.0001) for α = 20.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7889848947525024
Epoch time: 1.0239970684051514 seconds
###### Epoch 2 ######
(ε = 0.95, δ = 0.0001) for α = 17.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7890015840530396
Epoch time: 1.0244340896606445 seconds
###### Epoch 3 ######
(ε = 1.12, δ = 0.0001) for α = 15.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.788514256477356
Epoch time: 1.0259323120117188 seconds
###### Epoch 4 ######
(ε = 1.27, δ = 0.0001) for α = 14.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7868698835372925
Epoch time: 1.0249314308166504 seconds
###### Epoch 5 ######
(ε = 1.40, δ = 0.0001) for α = 13.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7838730812072754
Epoch time: 1.0249314308166504 seconds
###### Epoch 6 ######
(ε = 1.52, δ = 0.0001) for α = 12.0
Test Accuracy: 0.21844660194174756 ---- Test Loss: 1.775829553604126
Epoch time: 1.0259380340576172 seconds
###### Epoch 7 ######
(ε = 

A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(ε = 0.73, δ = 0.0001) for α = 20.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7893543243408203
Epoch time: 1.0993530750274658 seconds
###### Epoch 2 ######
(ε = 0.95, δ = 0.0001) for α = 17.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7898412942886353
Epoch time: 1.5692219734191895 seconds
###### Epoch 3 ######
(ε = 1.12, δ = 0.0001) for α = 15.0
Test Accuracy: 0.17475728155339806 ---- Test Loss: 1.790789008140564
Epoch time: 1.2231125831604004 seconds
###### Epoch 4 ######
(ε = 1.27, δ = 0.0001) for α = 14.0
Test Accuracy: 0.1796116504854369 ---- Test Loss: 1.7909345626831055
Epoch time: 1.0444550514221191 seconds
###### Epoch 5 ######
(ε = 1.40, δ = 0.0001) for α = 13.0
Test Accuracy: 0.18446601941747573 ---- Test Loss: 1.790610432624817
Epoch time: 1.2621479034423828 seconds
###### Epoch 6 ######
(ε = 1.52, δ = 0.0001) for α = 12.0
Test Accuracy: 0.17475728155339806 ---- Test Loss: 1.784796118736267
Epoch time: 1.1133837699890137 seconds
###### Epoch 7 ######
(ε =